# Using Q\# with Python

## Abstract

## The `qsharp` package

We begin by importing the `qsharp` Python package. This package provides Q# interoperability for Python, as well as integration with Jupyter.

In [1]:
import qsharp

Preparing Q# environment...


Once imported, the `qsharp` package exposes Q# functions and operations as Python object that we can use to simulate quantum programs, estimate quantum resources, and so forth.

Let's start by defining a new Q# operation with the `%%qsharp` magic command.

In [2]:
%%qsharp

operation SayHello(name : String) : Unit {
    Message($"Hello, {name}!");
}

This creates a new Python object `SayHello` that represents our new Q# operation.

In [3]:
SayHello

<Q# callable SayHello>

By using the `.simulate` method, we can run our new operation on a quantum simulator:

In [4]:
SayHello.simulate(name="quantum world")

Hello, quantum world!


()

Inputs to your Q# operation are represented by Python keyword arguments, and outputs are returned back to the Python host.

In [5]:
%%qsharp

function Plus(x : Int, y : Int) : Int {
    return x + y;
}

In [6]:
Plus.simulate(x=3, y=5)

8

Of course, you can also use qubits in the Q# programs that you simulate from Python:

In [7]:
%%qsharp
open Microsoft.Quantum.Measurement;

operation SampleQrng() : Result {
    use q = Qubit();
    H(q);
    return MResetZ(q);
}

In [8]:
SampleQrng.simulate()

1

The `qsharp` package also provides the `compile` function, which allows for compiling Q# code from Python strings.

In [9]:
sample_qrng = qsharp.compile("""
    operation SampleQrng() : Result {
        use q = Qubit();
        H(q);
        return MResetZ(q);
    }
""")

In [10]:
sample_qrng.simulate()

1

## Workspaces, Projects, and Packages


As your quantum programs get larger, it can be inconvienent to keep all of your Q# code in a single notebook or Python script. Instead, the `qsharp` package allows you to call into Q# code from source code in the same directory as your notebook.

For example, in the cell below, `PrepareBellPair` is defined in [`./Operations.qs`](./Operations.qs), but we can call it from Q# operations defined using `%%qsharp`.

In [20]:
%%qsharp
open Microsoft.Quantum.Samples;

operation PrepareAndMeasureBellPair() : (Result, Result) {
    use left = Qubit();
    use right = Qubit();

    PrepareBellPair(left, right);
    return (MResetZ(left), MResetZ(right));
}

In [19]:
PrepareAndMeasureBellPair.simulate()

(0, 0)


After importing the `qsharp` package, you can also import Q# namespaces as though they were Python packages. For example, `Operations.qs` also defines an operation that demonstrates how to run quantum teleportation; we can import it here and run the `RunTeleportationExample` operation on our simulator.

In [22]:
from Microsoft.Quantum.Samples import RunTeleportationExample

In [27]:
RunTeleportationExample.simulate()

Teleported successfully!


()

The Q# code in your workspace can also depend on other Q# _packages_ and _projects_ by using project files.

> **TIP**: If you don't have a project file for your workspace, the `qsharp` package will assume some reasonable defaults. Having a project file makes it easy to use additional packages, to get code completion and hover documentation while you edit your Q# files, and so forth.

For example, this sample includes a Q# project file that tells the `qsharp` Python package what other Q# packages are needed to run your code.

In [31]:
with open('python.csproj') as f:
    print(f.read())

<!-- This line declare that the project is a Q# project, built using the
     Microsoft.Quantum.Sdk package. -->
<Project Sdk="Microsoft.Quantum.Sdk/0.18.2107153439">

  <PropertyGroup>
    <OutputType>Library</OutputType>
    <TargetFramework>netstandard2.1</TargetFramework>
    <!-- This property tells the qsharp package to automatically load
         your project and use it as a workspace. -->
    <IQSharpLoadAutomatically>true</IQSharpLoadAutomatically>
  </PropertyGroup>

  <!-- To include functions and operations from other Q# packages, you can
       add them as package references in your project file. These packages
       will automatically be downloaded from nuget.org and made available to your
       Q# workspace. -->
  <ItemGroup>
    <PackageReference Include="Microsoft.Quantum.Numerics" Version="0.18.2107153439" />
  </ItemGroup>

</Project>



To see what packages are currently added to your workspace, you can use the `qsharp.packages` object:

In [35]:
qsharp.packages

Adding package .

[('Microsoft.Quantum.Standard', LooseVersion ('0.18.2107153439')), ('Microsoft.Quantum.Standard.Visualization', LooseVersion ('0.18.2107153439')), ('Microsoft.Quantum.Numerics', LooseVersion ('0.18.2107153439'))]

## Diagnostics

Just as with Q# standalone notebooks, you can also use diagnostics like `DumpMachine` and `DumpOperation` from Python notebooks to learn how your Q# program work and to help diagnose issues and bugs in your Q# programs.

In [36]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpPlusState() : Unit {
    use q = Qubit();
    within {
        H(q);
    } apply {
        DumpMachine();
    }
}

In [37]:
DumpPlusState.simulate()

|0⟩	0.7071067811865476 + 0𝑖
|1⟩	0.7071067811865476 + 0𝑖

()

The Q# package also allows you to capture these diagnostics and manipulate them as Python objects:

In [38]:
with qsharp.capture_diagnostics() as diagnostics:
    DumpPlusState.simulate()

In [39]:
diagnostics

[{'diagnostic_kind': 'state-vector',
  'div_id': 'dump-machine-div-7d5c6c6f-9344-48a6-9e47-c05342adbc07',
  'qubit_ids': [0],
  'n_qubits': 1,
  'amplitudes': [{'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0},
   {'Real': 0.7071067811865476,
    'Imaginary': 0.0,
    'Magnitude': 0.7071067811865476,
    'Phase': 0.0}]}]

Working with raw JSON for diagnostics can be somewhat inconvienent, so the `capture_diagnostics` function also supports converting diagnostics into quantum objects using the QuTiP library:

In [40]:
with qsharp.capture_diagnostics(as_qobj=True) as diagnostics:
    DumpPlusState.simulate()

In [42]:
diagnostics[0]

Quantum object: dims = [[2], [1]], shape = (2, 1), type = ket
Qobj data =
[[0.70710678]
 [0.70710678]]

In [47]:
%%qsharp
open Microsoft.Quantum.Diagnostics;

operation DumpOneState() : Unit {
    use q = Qubit();
    within {
        X(q);
    } apply {
        DumpMachine();
    }
}

In [49]:
with qsharp.capture_diagnostics() as diagnostics:
    DumpOneState.toffoli_simulate()

Offset  	State Data
========	==========
00000000	1


## Using other simulators

In [51]:
dir(qsharp)

['Any',
 'Config',
 'Dict',
 'IQSharpError',
 'List',
 'LooseVersion',
 'Packages',
 'Pauli',
 'Projects',
 'QSharpCallable',
 'QSharpModuleFinder',
 'Result',
 'Union',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 '_experimental_versions',
 '_start_client',
 'capture_diagnostics',
 'client',
 'clients',
 'compile',
 'component_versions',
 'config',
 'contextmanager',
 'defaultdict',
 'get_available_operations',
 'get_available_operations_by_namespace',
 'get_workspace_operations',
 'ipython_magic',
 'loader',
 'packages',
 'pkg_resources',
 'projects',
 'qobj',
 'qsharp',
 'reload',
 'serialization',
 'sys',
 'types',
 'version']